<a href="https://colab.research.google.com/github/hemanths03/CS6910_Assignment_3/blob/main/No_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np
from random import sample

from tensorflow import keras

#WandB Login

In [2]:
!pip install wandb -qqq
import wandb
wandb.login()

from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 144 kB 46.2 MB/s 
     |████████████████████████████████| 181 kB 36.2 MB/s 
     |████████████████████████████████| 63 kB 535 kB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


#Mounting the google drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Load the datasets

In [4]:
train_dataset = pd.read_csv('/content/gdrive/MyDrive/dakshina_dataset_v1.0/te/lexicons/te.translit.sampled.train.tsv', sep = '\t', header = None)
val_dataset = pd.read_csv('/content/gdrive/MyDrive/dakshina_dataset_v1.0/te/lexicons/te.translit.sampled.dev.tsv', sep = '\t', header = None)
test_dataset = pd.read_csv('/content/gdrive/MyDrive/dakshina_dataset_v1.0/te/lexicons/te.translit.sampled.test.tsv', sep = '\t', header = None)

#Preprocessing the dataset into required format

In [5]:
def preprocess_data():

    #Declaring some required variables

    train_input_lexicons = []
    train_target_lexicons = []
    val_input_lexicons = []
    val_target_lexicons = []
    test_input_lexicons = []
    test_target_lexicons = []

    #Splitting the datasets in input_lexicons and target_lexicons

    for i in range(len(train_dataset)):
      train_input_lexicons.append(str(train_dataset[1][i]))
      train_target_lexicons.append("\t" + str(train_dataset[0][i])+"\n")

    for i in range(len(val_dataset)):
      val_input_lexicons.append(str(val_dataset[1][i]))
      val_target_lexicons.append("\t" + str(val_dataset[0][i])+"\n")

    for i in range(len(test_dataset)):
      test_input_lexicons.append(str(test_dataset[1][i]))
      test_target_lexicons.append("\t" + str(test_dataset[0][i])+"\n")


    #Creating unique input and target character sets

    input_characters = set()
    input_characters.add(' ')
    target_characters = set()
    target_characters.add(' ')

    #union of all input words

    ip_words = train_input_lexicons + val_input_lexicons + test_input_lexicons

    #Union of all target words

    op_words = train_target_lexicons + val_target_lexicons + test_target_lexicons

    #Adding unique characters in their respective sets

    for word in ip_words:
      for char in word:
        input_characters.add(char)

    for word in op_words:
      for char in word:
        target_characters.add(char)

    #Sorting the list

    input_characters = sorted(list(input_characters))
    num_encoder_tokens = len(input_characters)

    target_characters = sorted(list(target_characters))
    num_decoder_tokens = len(target_characters)

    #Find the max sequence length input and target

    max_encoder_seq_length = max([len(word) for word in ip_words])
    max_decoder_seq_length = max([len(word) for word in op_words])

    #Printing the summary :

    print("Summary of the dataset :")
    print("Number of train samples :" , len(train_input_lexicons))
    print("Number of val samples :" , len(val_input_lexicons))
    print("Number of test samples :" , len(test_input_lexicons))
    print("Number of unique input tokens :" , num_encoder_tokens)
    print("Number of unique output tokens :" , num_decoder_tokens)
    print("Max sequence length for inputs:" , max_encoder_seq_length)
    print("Max sequence length for outputs:" , max_decoder_seq_length)

    #Creating a dictionary for input words and target words

    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])


    encoder_train_input_data = np.zeros((len(train_input_lexicons), max_encoder_seq_length), dtype="float32")

    encoder_val_input_data = np.zeros((len(val_input_lexicons), max_encoder_seq_length), dtype="float32")

    encoder_test_input_data = np.zeros((len(test_input_lexicons), max_encoder_seq_length), dtype="float32")


    decoder_train_input_data = np.zeros((len(train_input_lexicons), max_decoder_seq_length), dtype="float32")
    decoder_train_target_data = np.zeros((len(train_input_lexicons), max_decoder_seq_length, num_decoder_tokens ), dtype="float32")

   
    decoder_val_input_data = np.zeros((len(val_input_lexicons), max_decoder_seq_length), dtype="float32")
    decoder_val_target_data = np.zeros((len(val_input_lexicons), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

    
    #TRAIN DATA

    for i, (input_text, target_text) in enumerate(zip(train_input_lexicons, train_target_lexicons)):
      for t, char in enumerate(input_text):
          encoder_train_input_data[i, t] = input_token_index[char]
      encoder_train_input_data[i, t + 1 :] = input_token_index[' ']
      for t, char in enumerate(target_text):
          # decoder_target_data is ahead of decoder_input_data by one timestep
          decoder_train_input_data[i, t] = target_token_index[char]
          if t > 0:
              # decoder_target_data will be ahead by one timestep
              # and will not include the start character.
              decoder_train_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_train_input_data[i, t + 1 :] = target_token_index[' ']
      decoder_train_target_data[i, t:, target_token_index[' ']] = 1.0

    #VALIDATION DATA

    for i, (input_text, target_text) in enumerate(zip(val_input_lexicons, val_target_lexicons)):
      for t, char in enumerate(input_text):
          encoder_val_input_data[i, t] = input_token_index[char]
      encoder_val_input_data[i, t + 1 :] = input_token_index[' ']
      for t, char in enumerate(target_text):
          # decoder_target_data is ahead of decoder_input_data by one timestep
          decoder_val_input_data[i, t] = target_token_index[char]
          if t > 0:
              # decoder_target_data will be ahead by one timestep
              # and will not include the start character.
              decoder_val_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_val_input_data[i, t + 1 :] = target_token_index[' ']
      decoder_val_target_data[i, t:, target_token_index[' ']] = 1.0

    #TEST DATA

    for i, input_text in enumerate(test_input_lexicons):
      for t, char in enumerate(input_text):
          encoder_test_input_data[i, t] = input_token_index[char]
      encoder_test_input_data[i, t + 1 :] = input_token_index[' ']
      

    inverse_input_token_index = dict((i, char) for char, i in input_token_index.items())
    inverse_target_token_index = dict((i, char) for char, i in target_token_index.items())

    return ((encoder_train_input_data , encoder_val_input_data , encoder_test_input_data),
    (decoder_train_input_data , decoder_val_input_data),
    (decoder_train_target_data , decoder_val_target_data),
    (val_input_lexicons , test_input_lexicons),
    (val_target_lexicons , test_target_lexicons),
    (num_encoder_tokens , num_decoder_tokens),
    (max_encoder_seq_length , max_decoder_seq_length),
    (target_token_index , inverse_input_token_index , inverse_target_token_index))

#Load the preprocessed data

In [6]:
((encoder_train_input_data , encoder_val_input_data , encoder_test_input_data),
 (decoder_train_input_data , decoder_val_input_data),
 (decoder_train_target_data , decoder_val_target_data),
 (val_input_lexicons , test_input_lexicons),
 (val_target_lexicons , test_target_lexicons),
 (num_encoder_tokens , num_decoder_tokens),
 (max_encoder_seq_length , max_decoder_seq_length),
 (target_token_index , inverse_input_token_index , inverse_target_token_index)) = preprocess_data()

Summary of the dataset :
Number of train samples : 58550
Number of val samples : 5683
Number of test samples : 5747
Number of unique input tokens : 27
Number of unique output tokens : 66
Max sequence length for inputs: 25
Max sequence length for outputs: 22


#Building an RNN Model

In [7]:
def train():

    np.random.seed(77)

    #Initializing WandB
    run = wandb.init()
    config = run.config

    #Setting up the Run name
    name = "ES_" + str(config.embedding_size) + "_CT_" + config.cell_type + "_DO_" + str(config.dropout) + "_BS_" + str(config.beam_size)
    run.name = name

    #Set the variables
    enc_latent_dims = [config.hidden_layer_size] * config.encoder_layers
    dec_latent_dims  = [config.hidden_layer_size] * config.decoder_layers

    #Define input sequence and Setting the Encoder
    encoder_inputs = keras.Input(shape = (None, ))
    encoder_outputs = keras.layers.Embedding(input_dim = num_encoder_tokens, output_dim = config.embedding_size, input_length = max_encoder_seq_length)(encoder_inputs)

    #Setting the Decoder
    decoder_inputs = keras.Input(shape=(None, ))
    decoder_outputs = keras.layers.Embedding(input_dim = num_decoder_tokens, output_dim = config.embedding_size, input_length = max_decoder_seq_length)(decoder_inputs)

    # We discard encoder_outputs and only keep the states.
    # Set up the decoder, using encoder_states as initial state.
    encoder_states = list()

    for i in range(len(enc_latent_dims)):
        if config.cell_type == "rnn":
            encoder_outputs, state = keras.layers.SimpleRNN(enc_latent_dims[i], dropout = config.dropout, return_state = True, recurrent_dropout = config.recurrent_dropout, return_sequences = True)(encoder_outputs)
            encoder_states = [state]
        if config.cell_type == "lstm":
            encoder_outputs, state_h, state_c = keras.layers.LSTM(enc_latent_dims[i], dropout = config.dropout, return_state = True, recurrent_dropout = config.recurrent_dropout, return_sequences = True)(encoder_outputs)
            encoder_states = [state_h,state_c]
        if config.cell_type == "gru":
            encoder_outputs, state = keras.layers.GRU(enc_latent_dims[i], dropout = config.dropout, return_state = True, recurrent_dropout = config.recurrent_dropout, return_sequences = True)(encoder_outputs)
            encoder_states = [state]

    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    decoder_states = encoder_states.copy()

    for j in range(len(dec_latent_dims)):
        if config.cell_type == "rnn":
            decoder = keras.layers.SimpleRNN(dec_latent_dims[i], dropout = config.dropout, return_sequences = True, recurrent_dropout = config.recurrent_dropout, return_state = True)
            decoder_outputs, state = decoder(decoder_outputs, initial_state = decoder_states)
        if config.cell_type == "lstm":
            decoder = keras.layers.LSTM(dec_latent_dims[i], dropout = config.dropout, return_sequences = True, recurrent_dropout = config.recurrent_dropout, return_state = True)
            decoder_outputs, state_h, state_c = decoder(decoder_outputs, initial_state = decoder_states)
        if config.cell_type == "gru":
            decoder = keras.layers.GRU(dec_latent_dims[i], dropout = config.dropout, return_sequences = True, recurrent_dropout = config.recurrent_dropout, return_state = True)
            decoder_outputs, state = decoder(decoder_outputs, initial_state = decoder_states)
    
    #Adding a dense layer
    decoder_dense = keras.layers.Dense(num_decoder_tokens, activation = "softmax")
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn on
    # encoder_train_input_data & decoder_train_input_data into decoder_train_target_data

    model = keras.Model([encoder_inputs , decoder_inputs] , decoder_outputs)

    #Compiling the model
    model.compile(optimizer="adam" , loss = "categorical_crossentropy" , metrics=["accuracy"])

    #Fitting the model
    model.fit(
        [encoder_train_input_data, decoder_train_input_data],
        decoder_train_target_data,
        batch_size = config.batch_size,
        epochs = config.epochs,
        callbacks = [WandbCallback()]
    )

    #save the model
    model.save("seq2seq")

    #Calculating validation accuracy using inference on validation data
    #val_accuracy = infer(encoder_val_input_data, val_input_lexicons, val_target_lexicons, num_decoder_tokens, max_decoder_seq_length, target_token_index, inverse_target_token_index, enc_latent_dims, dec_latent_dims, config.cell_type, config.beam_size)
    #print("VALACCC",val_accuracy)
    #wandb.log( { "val_accuracy": val_accuracy})

    #Calculating test accuracy using inference on test data
    #test_accuracy = infer(encoder_test_input_data, test_input_words, test_target_words, num_decoder_characters, max_decoder_seq_length, target_characters_index, inverse_target_characters_index, enc_latent_dims, dec_latent_dims, config.cell_type, config.beam_size)
    #print("TEST",test_accuracy)
    # wandb.log( { "test_accuracy": test_accuracy} )


#Sweep Configuration

In [8]:
sweep_config_temp = {
  "name": "sweep-1",

  "method": "grid",

  "metric": {
      "name": "accuracy",
      "goal": "maximize"  
    },
    
  "parameters": {
        "embedding_size": {
            "values": [32 , 64 , 256]
        },
        "encoder_layers" :{
            "values" : [1 , 2 , 3]
        },
        "decoder_layers": {
            "values": [1 , 2 , 3]
        },
        "hidden_layer_size": {
            "values": [64 , 256]
        },
        "cell_type": {
            "values": ["rnn" , "lstm" , "gru"]
        },
        "dropout": {
            "values": [0.2 , 0.3]
        },
        "beam_size": {
            "values": [0]
        },
        "recurrent_dropout": {
            "values": [0.0 , 4.0 , 6.0]
        },
        "batch_size": {
            "values": [128]
        },
        "epochs": {
            "values": [25]
        }
    }
}

In [9]:
sweep_id = wandb.sweep(sweep_config_temp, entity="cs21m027_cs21m011", project="DL_ASG_3_temp")
wandb.agent(sweep_id, train)

Create sweep with ID: d4zn06mq
Sweep URL: https://wandb.ai/cs21m027_cs21m011/DL_ASG_3_temp/sweeps/d4zn06mq


wandb: Agent Starting Run: xmtxz4ti with config:
wandb: 	batch_size: 128
wandb: 	beam_size: 0
wandb: 	cell_type: gru
wandb: 	decoder_layers: 1
wandb: 	dropout: 0.2
wandb: 	embedding_size: 256
wandb: 	encoder_layers: 1
wandb: 	epochs: 25
wandb: 	hidden_layer_size: 256
wandb: 	recurrent_dropout: 0
wandb: Currently logged in as: cs21m027_cs21m011 (use `wandb login --relogin` to force relogin)


Epoch 1/25
458/458 [==============================] - 24s 27ms/step - loss: 1.1557 - accuracy: 0.6931 - _timestamp: 1650284767.0000 - _runtime: 36.0000
Epoch 2/25
458/458 [==============================] - 13s 27ms/step - loss: 0.8656 - accuracy: 0.7521 - _timestamp: 1650284780.0000 - _runtime: 49.0000
Epoch 3/25
458/458 [==============================] - 13s 27ms/step - loss: 0.6859 - accuracy: 0.7979 - _timestamp: 1650284792.0000 - _runtime: 61.0000
Epoch 4/25
458/458 [==============================] - 12s 27ms/step - loss: 0.4682 - accuracy: 0.8619 - _timestamp: 1650284805.0000 - _runtime: 74.0000
Epoch 5/25
458/458 [==============================] - 12s 27ms/step - loss: 0.3080 - accuracy: 0.9095 - _timestamp: 1650284817.0000 - _runtime: 86.0000
Epoch 6/25
458/458 [==============================] - 12s 27ms/step - loss: 0.2197 - accuracy: 0.9353 - _timestamp: 1650284830.0000 - _runtime: 99.0000
Epoch 7/25
458/458 [==============================] - 13s 28ms/step - loss: 0.1696 - acc

INFO:tensorflow:Assets written to: seq2seq/assets


INFO:tensorflow:Assets written to: seq2seq/assets


accuracy,▁▂▃▅▆▇▇▇▇████████████████
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▆▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.99181
epoch,24
loss,0.02648


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
